In [182]:
#On entre un élément en entrée

mot=['moutarde']

In [183]:
sc

In [184]:
spark.version

u'2.1.0'

In [185]:
# -*- coding: utf-8 -*-
import numpy as np
import datetime as tm
import json
import pandas as pd
from pyspark.storagelevel import StorageLevel
from pyspark.sql import *
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
from pyspark import SparkConf, SparkContext, HiveContext
from pyspark.sql.window import Window
import pyspark.sql.functions as F
import cPickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
from tr_utils import *

adv = 656228
dates = get_period('2017-06-07', '2017-06-14')
path = 'gs://tr-parquet/its_panel/'
dfs = []
for date in dates:
    dfs += [sqlContext.read.load(path+date+'/ad='+str(adv))]
    
data = reduce(lambda x, y: x.unionAll(y), dfs)



In [186]:
#On récupère les données sous forme d'un DataFrame
dl = data.filter((F.col('ty')=='datalayer')&(F.col('id2')!='')&(F.col('id2')!=-1)&(F.col('id2')!=0))

test = spark.read.json(dl.limit(1000).rdd.map(lambda x: x.dl))
schema = test.schema
with open('schema_dl_jv.pickle', 'w') as f:
    cPickle.dump(schema, f)
    
dl_parsed = dl.withColumn('dl2', F.from_json('dl', schema))

In [187]:
dl_parsed.printSchema()

In [188]:
#On sélectionne les identifiants des users, les recettes consultées ainsi que les ingrédients associés aux recettes
#Les ingrédients ne sont pas donnés comme des mots clés, mais avec la quantité associée (sous forme de phrase..)
df=dl_parsed.select('dl2.ses.uuid2','dl2.dataLayer.sectionData.averageScore','dl2.dataLayer.sectionData.title','dl2.dataLayer.sectionData.difficulty',
                    'dl2.dataLayer.sectionData.cost','dl2.dataLayer.sectionData.ingredients','dl2.dataLayer.sectionData.tags','dl2.dataLayer.sectionData.time')

In [180]:
df.show(truncate=False)

In [181]:
def setup(u):
    if u!=None:
        if u[1]!=None:
            return float(u[1])
        else:
            return 0.0
    else:
        return None

In [10]:
setup_=F.udf(setup,FloatType())

In [11]:
def cooking(u):
    if u!=None:
        if u[0]!=None:
            return float(u[0])
        else:
            return 0.0
    else:
        return None

In [12]:
cooking_=F.udf(cooking,FloatType())

In [13]:
df=df.select('uuid2','averageScore','title','difficulty',
                    'cost','ingredients','tags','time',setup_('time').alias('setup'),cooking_('time').alias('cooking'))

In [15]:
#On regroupe dans une même ligne toutes les données relatives à un user: recettes et ingrédients associés à ces recettes
from pyspark.sql.functions import collect_list
df=df.dropDuplicates()
grouped = df.groupBy(['uuid2']).agg(collect_list("title").alias('listerecettes'),collect_list("ingredients").alias('ingredientsrecettes'),
                                    collect_list("difficulty").alias('listedifficulty'),collect_list("cost").alias('listecost'),collect_list("tags").alias('listetags'),
                                    collect_list("averageScore").alias('NotesMoyennes'),collect_list("setup").alias('listesetup'),
                                    collect_list("cooking").alias('listecooking'))


In [16]:
grouped.head()

Row(uuid2=u'1005488911830817536', listerecettes=[u'Verrine fraises et biscuits'], ingredientsrecettes=[u'1 c a s bombee de fructose;30g de biscuits emiettes;10 cl de creme liquide 4 de m g;1 yaourt nature 0 m g;250g de ricotta;150g de fraises'], listedifficulty=[2], listecost=[2], listetags=[Row(primary=[u'verrines_de_fraises', u'verrines_aux_biscuits', u'verrines_de_fruits_rouges', u'verrines_de_fruits', u'verrines_de_fraises_et_biscuits', u'fraises', u'biscuits', u'desserts', u'gouter'], secondary=[u'verrine_de_fraises', u'verrine_de_biscuits', u'verrine_de_fruits_rouges', u'verrine_de_fruits', u'verrine_fraise_biscuit', u'fraises', u'biscuits', u'desserts', u'gouters'])], NotesMoyennes=[4.3], listesetup=[10.0], listecooking=[0.0])

In [17]:
#on veut construire une liste contenant tous les mots de la liste d'ingrédients, en se débarassant dans un premier temps 
#des quantités, puis des stopwords à l'aide de la commande existante du package NLTK.
#On se débarasse également des lettres seules exclues, provenant d'abréviations comme c a s pour 'cuillière à soupe', 
#g pour grammes, mg, kg etc.

import re
from nltk.corpus import stopwords
from pyspark.ml.feature import StopWordsRemover

list_nb = '0 1 2 3 4 5 6 7 8 9'.split()
list_lt='a b c d e f g h i j k l m n o p q r s t u v w kg mg gr cl ml dl cuil'.split()
stopwords=StopWordsRemover.loadDefaultStopWords('french')

def transform (l):
    return [mot for recette in l for mot in re.split(' |;',''.join([i if i not in list_nb else '' for i in recette ])) if mot!='' if mot not in stopwords and mot not in list_lt] 

In [18]:
transform_df  = F.udf(transform,ArrayType(StringType()))

In [19]:
grouped2=grouped.select('uuid2','listerecettes',transform_df('ingredientsrecettes').alias('ingredientsrecettes2'),'listedifficulty',
                        'listecost','listetags','NotesMoyennes','listesetup','listecooking')

In [21]:
#On calcule, pour chaque user, la moyenne des notes, coûts, temps de préparation, scores de difficulté des recettes qu'il a consultés
def moyenne(u):
    try:
        l=[float(int(i))for i in u]
        return float(int(float(sum(l))/float(len(l))))
    except:
        return None

In [22]:
moyenne_=F.udf(moyenne,FloatType())

In [23]:
grouped2=grouped2.select('uuid2','listerecettes','ingredientsrecettes2',moyenne_('listedifficulty').alias('difficulty'),
                         moyenne_('listecost').alias('cost'),'listetags',moyenne_('listesetup').alias('setup'),moyenne_('listecooking').alias('cooking'),
                         moyenne_('NotesMoyennes').alias('note'))

In [25]:
#On sélectionne les individus qui ont visionné au moins une recette contenant l'ingrédient entré au début du notebook
def isin(u):
    for m in mot:
        if m in u:
            return 1.0
            break
        else:
            return 0.0

In [26]:
isin_=F.udf(isin,FloatType())

In [27]:
groupedmot=grouped2.select('uuid2','listerecettes','ingredientsrecettes2',isin_('ingredientsrecettes2').alias('isin'),
                              'difficulty','cost','listetags','note','setup','cooking')

In [28]:
groupedmot=groupedmot.filter(groupedmot.isin==1.0)

In [ ]:
#Entrée à la main de grands groupes d'aliments dont on désire voir ou non une surreprésentation dans les consultations de recettes de
#nos consommateurs de l'ingrédient entré au début du notebook

In [30]:
listefruits=['abricot','ananas','banane','cassis','cerise','chataigne',
'citron',
'clémentine',
'coing',
'datte',
'figue',
'fraise',
'framboise',
'passion',
'grenade',
'groseille',
'kaki',
'kiwi',
'kumquat',
'kitchi',
'mandarine',
'mangue',
'marron',
'melon',
'mirabelle',
'mure',
'myrtille',
'nectarine',
'noisette',
'noix',
'orange',
'pamplemousse',
'papaye',
'pastèque',
'pêche',
'poire',
'pomme',
'prune',
'quetsche',
'raisin',
'reine-claude']

In [31]:
listelegumes= ['ail',
'artichaut',
'asperge',
'aubergine',
'bette',
'betterave',
'brocoli',
'carotte',
'catalonia',
'celeri',
'chou' ,
'citrouille',
'concombre',
'courge',
'courgette',
'cresson',
'endive',
'epinard',
'fenouil',
'haricot',
'laitue',
'mache',
'mais',
'navet',
'oignon',
'panais',
'pois',
'poireau',
'poivron',
'pomme de terre',
'potimarron',
'potiron',
'radis',
'salsifis',
'topinambour',
'avocat',
'tomate']

In [32]:
listeviandes=['boeuf',
'agneau',
'cheval',
'mouton',
'abats',
'gibier',
'porc',
'veau',
'lapin',
'volaille',
'gibier',
'jambon',
'dinde',
'bacon',
'poulet']

In [33]:
listepoissons=['anchois',
'anguille',
'bar',
'barbue',
'baudroie',
'bonite',
'cabillaud',
'morue',
'colin',
'dorade',
'espadon',
'haddock',
'hareng',
'lamproie',
'lieu',
'lotte',
'maquereau',
'merlan',
'merlu',
'merou',
'mulet',
'rouget',
'requin',
'sabre',
'saumon',
'saumonette',
'sole',
'thon',
'truite',
'turbot']

In [34]:
#analyse viande

In [35]:
#Fonction qui définit si l'individu a visité une recette contenant une viande
def isinviande(u):
    for i in listeviandes:
        if i in u:
            return 1.0
            break
        else:
            return 0.0

In [36]:
isinviande_=F.udf(isinviande,FloatType())

In [37]:
groupedviande1=groupedmot.select('uuid2','listerecettes','ingredientsrecettes2',isinviande_('ingredientsrecettes2').alias('isinviande'),
                              'difficulty','cost','listetags','note','setup','cooking')

In [38]:
groupedviande1.show()

+-------------------+--------------------+--------------------+----------+----------+----+--------------------+----+-----+-------+
|              uuid2|       listerecettes|ingredientsrecettes2|isinviande|difficulty|cost|           listetags|note|setup|cooking|
+-------------------+--------------------+--------------------+----------+----------+----+--------------------+----+-----+-------+
|1168571936604417458|[Wrap au thon et ...|[huile, olive, mo...|       0.0|       2.0| 1.0|[[WrappedArray(wr...| 3.0| 15.0|    0.0|
|1241675771174869980|[Salade fa�on Ca�...|[vinaigre, blanc,...|       0.0|       2.0| 1.0|[[WrappedArray(sa...| 4.0| 20.0|   10.0|
|1247862988755582166|[Tarte au thon et...|[fromage, rape, g...|       0.0|       1.0| 1.0|[[WrappedArray(ta...| 4.0|  5.0|   30.0|
|1350591312933458002|[Pain de viande f...|[table, cassonade...|       1.0|       2.0| 2.0|[[WrappedArray(pa...| 4.0|  0.0|    0.0|
|1376558862739121882|[Escargots � la s...|[piment, cayenne,...|       0.0|       1.

In [39]:
#Nombre de personnes qui ont visité au moins une recette contenant le mot en entrée et au moins une recette contenant une viande
gr1=groupedviande1.filter(groupedviande1.isinviande==1.0).count()

In [41]:
gr1

2609

In [42]:
#Nombre de personnes qui ont visité au moins une recette contenant le mot en entrée et aucune recette contenant une viande
gr2=groupedmot.count()-gr1

In [43]:
gr2

37230

In [44]:
#Pourcentage associé au groupe "viande" de l'échantillon test (pourcentage d'individus qui ont visité au moins une recette de viande),
#celui qui contient les individus consommateurs de l'ingrédient entré au début du notebook
pourcentage1=float(gr1)/float(gr1+gr2)

In [45]:
pourcentage1

0.06548859158111399

In [46]:
groupedviande2=grouped2.select('uuid2','listerecettes','ingredientsrecettes2',isinviande_('ingredientsrecettes2').alias('isinviande'),
                               'difficulty','cost','listetags','note','setup','cooking')

In [47]:
gr3=groupedviande2.filter(groupedviande2.isinviande==1.0).count()

In [48]:
gr3

17619

In [49]:
gr4=grouped2.count()-gr1

In [50]:
gr4

482637

In [51]:
#Pourcentage associé au groupe "viande" de l'échantillon témoin (pourcentage d'individus qui ont visité au moins une recette de viande),
#celui qui contient tous les individus
pourcentage2=float(gr3)/float(gr3+gr4)

In [52]:
pourcentage2

0.035219967376703125

In [ ]:
#On procède de même pour tous les groupes que l'on a sélectionné en amont

In [53]:
#analyse poisson

In [54]:
def isinpoisson(u):
    for i in listepoissons:
        if i in u:
            return 1.0
            break
        else:
            return 0.0

In [55]:
isinpoisson_=F.udf(isinpoisson,FloatType())

In [56]:
groupedpoisson1=groupedmot.select('uuid2','listerecettes','ingredientsrecettes2',isinpoisson_('ingredientsrecettes2').alias('isinpoisson'),
                               'difficulty','cost','listetags','note','setup','cooking')

In [59]:
gr5=groupedpoisson1.filter(groupedpoisson1.isinpoisson==1.0).count()

In [60]:
gr5

1305

In [61]:
gr6=groupedmot.count()-gr1

In [62]:
gr6

37230

In [63]:
pourcentage3=float(gr5)/float(gr6+gr5)

In [64]:
pourcentage3

0.03386531724406384

In [65]:
groupedpoisson2=grouped2.select('uuid2','listerecettes','ingredientsrecettes2',isinpoisson_('ingredientsrecettes2').alias('isinpoisson'),
                               'difficulty','cost','listetags','note','setup','cooking')

In [66]:
gr7=groupedpoisson2.filter(groupedpoisson2.isinpoisson==1.0).count()

In [67]:
gr7

4398

In [68]:
gr8=grouped2.count()-gr1

In [69]:
gr8

482637

In [70]:
pourcentage4=float(gr7)/float(gr7+gr8)

In [71]:
pourcentage4

0.00903015183713696

In [72]:
#analyse legume

In [73]:
def isinlegume(u):
    for i in listelegumes:
        if i in u:
            return 1.0
            break
        else:
            return 0.0

In [74]:
isinlegume_=F.udf(isinlegume,FloatType())

In [75]:
groupedlegume1=groupedmot.select('uuid2','listerecettes','ingredientsrecettes2',isinlegume_('ingredientsrecettes2').alias('isinlegume'),
                               'difficulty','cost','listetags','note','setup','cooking')

In [77]:
gr9=groupedlegume1.filter(groupedlegume1.isinlegume==1.0).count()

In [78]:
gr9

12698

In [79]:
gr10=groupedmot.count()-gr1

In [80]:
gr10

37230

In [81]:
pourcentage5=float(gr9)/float(gr9+gr10)

In [82]:
pourcentage5

0.25432622977087005

In [83]:
groupedlegume2=grouped2.select('uuid2','listerecettes','ingredientsrecettes2',isinlegume_('ingredientsrecettes2').alias('isinlegume'),
                               'difficulty','cost','listetags','note','setup','cooking')

In [84]:
gr11=groupedlegume2.filter(groupedlegume2.isinlegume==1.0).count()

In [85]:
gr11

95510

In [86]:
gr12=grouped2.count()-gr1

In [87]:
pourcentage6=float(gr11)/float(gr11+gr12)

In [88]:
pourcentage6

0.1652001999491479

In [89]:
#analyse fruits


In [90]:
def isinfruit(u):
    for i in listefruits:
        if i in u:
            return 1.0
            break
        else:
            return 0.0

In [91]:
isinfruit_=F.udf(isinfruit,FloatType())

In [92]:
groupedfruit1=groupedmot.select('uuid2','listerecettes','ingredientsrecettes2',isinfruit_('ingredientsrecettes2').alias('isinfruit'),
                               'difficulty','cost','listetags','note','setup','cooking')

In [94]:
gr13=groupedfruit1.filter(groupedfruit1.isinfruit==1.0).count()

In [95]:
gr13

45

In [96]:
gr14=groupedmot.count()-gr1

In [97]:
gr14

37230

In [98]:
pourcentage7=float(gr13)/float(gr13+gr14)

In [99]:
pourcentage7

0.0012072434607645875

In [100]:
groupedfruit2=grouped2.select('uuid2','listerecettes','ingredientsrecettes2',isinfruit_('ingredientsrecettes2').alias('isinfruit'),
                               'difficulty','cost','listetags','note','setup','cooking')

In [101]:
gr15=groupedfruit2.filter(groupedfruit2.isinfruit==1.0).count()

In [102]:
gr15

1748

In [103]:
gr16=grouped2.count()-gr1

In [104]:
pourcentage8=float(gr15)/float(gr15+gr16)

In [105]:
pourcentage8

0.0036086996913612104

In [106]:
#analyse oeuf

In [107]:
def isinoeuf(u):
    if 'oeuf' in u:
        return 1.0
    else:
        return 0.0

In [108]:
isinoeuf_=F.udf(isinoeuf,FloatType())

In [109]:
groupedoeuf1=groupedmot.select('uuid2','listerecettes','ingredientsrecettes2',isinoeuf_('ingredientsrecettes2').alias('isinoeuf'),
                               'difficulty','cost','listetags','note','setup','cooking')

In [111]:
gr17=groupedoeuf1.filter(groupedoeuf1.isinoeuf==1.0).count()

In [112]:
gr17

8898

In [113]:
gr18=groupedmot.count()-gr1

In [114]:
gr18

37230

In [115]:
pourcentage9=float(gr17)/float(gr17+gr18)

In [116]:
pourcentage9

0.19289802289282

In [117]:
groupedoeuf2=grouped2.select('uuid2','listerecettes','ingredientsrecettes2',isinoeuf_('ingredientsrecettes2').alias('isinoeuf'),
                               'difficulty','cost','listetags','note','setup','cooking')

In [118]:
gr19=groupedoeuf2.filter(groupedoeuf2.isinoeuf==1.0).count()

In [119]:
gr19

76319

In [120]:
gr20=grouped2.count()-gr1

In [121]:
gr20

482637

In [122]:
pourcentage10=float(gr19)/float(gr19+gr20)

In [123]:
pourcentage10

0.13653847530038143

In [124]:
#analyse cocktail

def isincocktail(u):
    for i in u:
        for j in i:
            for k in j:
                if k.find('cocktail')!=-1:
                    return 1.0
                    break
                else:
                    return 0.0
    
    
isincocktail_=F.udf(isincocktail,FloatType())

groupedcocktail1=groupedmot.select('uuid2','listerecettes','ingredientsrecettes2','listetags','difficulty','setup','cooking',
                                   'cost',isincocktail_('listetags').alias('isincocktail'),'note')

gr21=groupedcocktail1.filter(groupedcocktail1.isincocktail==1.0).count()
gr22=groupedmot.count()-gr1
pourcentage11=float(gr21)/float(gr21+gr22)

In [125]:

groupedcocktail2=grouped2.select('uuid2','listerecettes','ingredientsrecettes2','setup','cooking',isincocktail_('listetags').alias('isincocktail'),
                              'difficulty','cost','listetags','note')
gr23=groupedcocktail2.filter(groupedcocktail2.isincocktail==1.0).count()
gr24=grouped2.count()-gr1

pourcentage12=float(gr23)/float(gr23+gr24)

In [126]:
# analyse aperitif

def isinaperitif(u):
    for i in u:
        for j in i:
            for k in j:
                if k.find('aperitif')!=-1:
                    return 1.0
                    break
                else:
                    return 0.0
    
    
isinaperitif_=F.udf(isinaperitif,FloatType())

groupedaperitif1=groupedmot.select('uuid2','listerecettes','ingredientsrecettes2','listetags','difficulty','setup','cooking',
                                   'cost',isinaperitif_('listetags').alias('isinaperitif'),'note')

gr25=groupedaperitif1.filter(groupedaperitif1.isinaperitif==1.0).count()
gr26=groupedmot.count()-gr1
pourcentage13=float(gr25)/float(gr25+gr26)

In [127]:

groupedaperitif2=grouped2.select('uuid2','listerecettes','ingredientsrecettes2','setup','cooking',isinaperitif_('listetags').alias('isinaperitif'),
                              'difficulty','cost','listetags','note')
gr27=groupedaperitif2.filter(groupedaperitif2.isinaperitif==1.0).count()
gr28=grouped2.count()-gr1

pourcentage14=float(gr27)/float(gr27+gr28)

In [128]:
#analyse entree

def isinentree(u):
    for i in u:
        for j in i:
            for k in j:
                if k.find('entree')!=-1:
                    return 1.0
                    break
                else:
                    return 0.0
    
    
isinentree_=F.udf(isinentree,FloatType())

groupedentree1=groupedmot.select('uuid2','listerecettes','ingredientsrecettes2','listetags','difficulty','setup','cooking',
                                   'cost',isinentree_('listetags').alias('isinentree'),'note')

gr29=groupedentree1.filter(groupedentree1.isinentree==1.0).count()
gr30=groupedmot.count()-gr1
pourcentage15=float(gr29)/float(gr29+gr30)

In [129]:

groupedentree2=grouped2.select('uuid2','listerecettes','ingredientsrecettes2','setup','cooking',isinentree_('listetags').alias('isinentree'),
                              'difficulty','cost','listetags','note')
gr31=groupedentree2.filter(groupedentree2.isinentree==1.0).count()
gr32=grouped2.count()-gr1

pourcentage16=float(gr31)/float(gr31+gr32)

In [130]:
#analyse plat

def isinplat(u):
    for i in u:
        for j in i:
            for k in j:
                if k.find('plat')!=-1:
                    return 1.0
                    break
                else:
                    return 0.0
    
    
    
isinplat_=F.udf(isinplat,FloatType())

groupedplat1=groupedmot.select('uuid2','listerecettes','ingredientsrecettes2','setup','cooking','listetags','difficulty',
                                   'cost',isinplat_('listetags').alias('isinplat'),'note')

gr33=groupedplat1.filter(groupedplat1.isinplat==1.0).count()
gr34=groupedmot.count()-gr1
pourcentage17=float(gr33)/float(gr33+gr34)

In [131]:

groupedplat2=grouped2.select('uuid2','listerecettes','ingredientsrecettes2','setup','cooking',isinplat_('listetags').alias('isinplat'),
                              'difficulty','cost','listetags','note')
gr35=groupedplat2.filter(groupedplat2.isinplat==1.0).count()
gr36=grouped2.count()-gr1

pourcentage18=float(gr35)/float(gr35+gr36)

In [132]:
#analyse dessert

def isindessert(u):
    for i in u:
        for j in i:
            for k in j:
                if k.find('dessert')!=-1:
                    return 1.0
                    break
                else:
                    return 0.0
    
    
isindessert_=F.udf(isindessert,FloatType())

groupeddessert1=groupedmot.select('uuid2','listerecettes','ingredientsrecettes2','listetags','difficulty','setup','cooking',
                                   'cost',isindessert_('listetags').alias('isindessert'),'note')

gr37=groupeddessert1.filter(groupeddessert1.isindessert==1.0).count()
gr38=groupedmot.count()-gr1
pourcentage19=float(gr37)/float(gr37+gr38)

In [133]:

groupeddessert2=grouped2.select('uuid2','listerecettes','setup','cooking','ingredientsrecettes2',isindessert_('listetags').alias('isindessert'),
                              'difficulty','cost','listetags','note')
gr39=groupeddessert2.filter(groupeddessert2.isindessert==1.0).count()
gr40=grouped2.count()-gr1

pourcentage20=float(gr39)/float(gr39+gr40)

In [134]:
#analyse sauce

def isinsauce(u):
    for i in u:
        for j in i:
            for k in j:
                if k.find('sauce')!=-1:
                    return 1.0
                    break
                else:
                    return 0.0
    
isinsauce_=F.udf(isinsauce,FloatType())

groupedsauce1=groupedmot.select('uuid2','listerecettes','ingredientsrecettes2','listetags','difficulty','setup','cooking',
                                   'cost',isinsauce_('listetags').alias('isinsauce'),'note')

gr41=groupedsauce1.filter(groupedsauce1.isinsauce==1.0).count()
gr42=groupedmot.count()-gr1
pourcentage21=float(gr41)/float(gr41+gr42)

In [136]:

groupedsauce2=grouped2.select('uuid2','listerecettes','setup','cooking','ingredientsrecettes2',isinsauce_('listetags').alias('isinsauce'),
                              'difficulty','cost','listetags','note')
gr43=groupedsauce2.filter(groupedsauce2.isinsauce==1.0).count()
gr44=grouped2.count()-gr1

pourcentage22=float(gr43)/float(gr43+gr44)

In [137]:
#analyse cost

gr45= groupedmot.filter(groupedmot.cost <=1).count()
gr46= groupedmot.filter(groupedmot.cost ==2).count()
gr47= groupedmot.filter(groupedmot.cost > 2).count()

pourcentage25=float(gr45)/float(groupedmot.count())
pourcentage26=float(gr46)/float(groupedmot.count())
pourcentage27=float(gr47)/float(groupedmot.count())

In [138]:

gr48= grouped2.filter(grouped2.cost <=1).count()
gr49 = grouped2.filter(grouped2.cost ==2).count()
gr50 = grouped2.filter(grouped2.cost >2).count()

pourcentage28=float(gr48)/float(grouped2.count())
pourcentage29=float(gr49)/float(grouped2.count())
pourcentage30=float(gr50)/float(grouped2.count())

In [139]:
#analyse difficulty

gr51= groupedmot.filter(groupedmot.difficulty <=1).count()
gr52= groupedmot.filter(groupedmot.difficulty ==2).count()
gr53= groupedmot.filter(groupedmot.difficulty >2).count()

pourcentage31=float(gr51)/float(groupedmot.count())
pourcentage32=float(gr52)/float(groupedmot.count())
pourcentage33=float(gr53)/float(groupedmot.count())

In [140]:
gr54= grouped2.filter(grouped2.difficulty <=1).count()
gr55= grouped2.filter(grouped2.difficulty ==2).count()
gr56= grouped2.filter(grouped2.difficulty >2).count()

pourcentage34=float(gr54)/float(grouped2.count())
pourcentage35=float(gr55)/float(grouped2.count())
pourcentage36=float(gr56)/float(grouped2.count())

In [141]:
#analyse note

gr57= groupedmot.filter(groupedmot.note <=3).count()
gr58= groupedmot.filter(groupedmot.note ==4).count()
gr59= groupedmot.filter(groupedmot.note>4).count()

pourcentage37=float(gr57)/float(groupedmot.count())
pourcentage38=float(gr58)/float(groupedmot.count())
pourcentage39=float(gr59)/float(groupedmot.count())

In [142]:
gr60= grouped2.filter(grouped2.note <=3).count()
gr61= grouped2.filter(grouped2.note ==4).count()
gr62= grouped2.filter(grouped2.note> 4).count()

pourcentage40=float(gr60)/float(grouped2.count())
pourcentage41=float(gr61)/float(grouped2.count())
pourcentage42=float(gr62)/float(grouped2.count())

In [143]:
#analyse setup

gr63= groupedmot.filter(groupedmot.setup <=10).count()
gr64= groupedmot.filter((10< groupedmot.setup) & (groupedmot.setup<=20)).count()
gr65= groupedmot.filter(groupedmot.setup >20).count()

pourcentage43=float(gr63)/float(groupedmot.count())
pourcentage44=float(gr64)/float(groupedmot.count())
pourcentage45=float(gr65)/float(groupedmot.count())

In [144]:
gr66= grouped2.filter(grouped2.setup <=10).count()
gr67= grouped2.filter((10< groupedmot.setup) & (groupedmot.setup<=20)).count()
gr68= grouped2.filter(grouped2.setup >=20).count()

pourcentage46=float(gr66)/float(grouped2.count())
pourcentage47=float(gr67)/float(grouped2.count())
pourcentage48=float(gr68)/float(grouped2.count())

In [145]:
#analyse cooking

gr69= groupedmot.filter(groupedmot.cooking <=10).count()
gr70= groupedmot.filter((10< groupedmot.cooking) & (groupedmot.cooking<=30)).count()
gr71= groupedmot.filter(groupedmot.cooking >=30).count()

pourcentage49=float(gr69)/float(groupedmot.count())
pourcentage50=float(gr70)/float(groupedmot.count())
pourcentage51=float(gr71)/float(groupedmot.count())

In [146]:
gr72= grouped2.filter(grouped2.cooking <=10).count()
gr73= grouped2.filter((10< groupedmot.cooking) & (groupedmot.cooking<=30)).count()
gr74= grouped2.filter(grouped2.cooking >=30).count()

pourcentage52=float(gr72)/float(grouped2.count())
pourcentage53=float(gr73)/float(grouped2.count())
pourcentage54=float(gr74)/float(grouped2.count())

In [147]:
import pandas
import numpy

ar = numpy.array([[pourcentage1, pourcentage2,gr1,gr2,gr3,gr4], [pourcentage3, pourcentage4,gr5,gr6,gr7,gr8], [pourcentage5, pourcentage6,gr9,gr10,gr11,gr12],
                  [pourcentage7, pourcentage8,gr13,gr14,gr15,gr16],[pourcentage9, pourcentage10,gr17,gr18,gr19,gr20],[pourcentage11, pourcentage12,gr21,gr22,gr23,gr24],
                  [pourcentage13, pourcentage14,gr25,gr26,gr27,gr28],[pourcentage15, pourcentage16,gr29,gr30,gr31,gr32],
                 [pourcentage17, pourcentage18,gr33,gr34,gr35,gr36],[pourcentage19, pourcentage20,gr37,gr38,gr39,gr40],[pourcentage21, pourcentage22,gr41,gr42,gr43,gr44],
                  [pourcentage25, pourcentage28,gr45,gr2,gr48,gr4],[pourcentage26, pourcentage29,gr46,gr2,gr49,gr4],[pourcentage27, pourcentage30,gr47,gr2,gr50,gr4],
                  [pourcentage31, pourcentage34,gr51,gr2,gr54,gr4],
                 [pourcentage32, pourcentage35,gr52,gr2,gr55,gr4],[pourcentage33, pourcentage36,gr53,gr2,gr56,gr4],[pourcentage37, pourcentage40,gr57,gr2,gr60,gr4],
                  [pourcentage38, pourcentage41,gr58,gr2,gr61,gr4],[pourcentage39, pourcentage42,gr59,gr2,gr62,gr4],[pourcentage43, pourcentage46,gr63,gr2,gr66,gr4],
                 [pourcentage44, pourcentage47,gr64,gr2,gr67,gr4],[pourcentage45, pourcentage48,gr65,gr2,gr68,gr4],[pourcentage49, pourcentage52,gr69,gr2,gr72,gr4],
                 [pourcentage50, pourcentage53,gr70,gr2,gr73,gr4],[pourcentage51, pourcentage54,gr71,gr2,gr74,gr4]])
df = pandas.DataFrame(ar, index = ['viande', 'poisson', 'legume','fruit','oeuf','cocktail','aperitif','entree','plat','dessert',
                                   'sauce','peucher','prixmoyen','cher','peudifficile','difficultemoyenne','difficile',
                                  'mauvaisenote','notemoyenne','bonnenote','peupreparation','preparationmoyenne','bcppreparation','peucuisson','cuissonmoyenne',
                                   'bcpcuisson'], columns = ['test', 'temoin','test&category','nbtest','temoin&category','nbtemoin'])

In [148]:
df

,test,temoin,test&category,nbtest,temoin&category,nbtemoin
viande,0.065489,0.035220,2609,37230,17619,482637
poisson,0.033865,0.009030,1305,37230,4398,482637
legume,0.254326,0.165200,12698,37230,95510,482637
fruit,0.001207,0.003609,45,37230,1748,482637
oeuf,0.192898,0.136538,8898,37230,76319,482637
cocktail,0.001609,0.004458,60,37230,2161,482637
aperitif,0.007703,0.004006,289,37230,1941,482637
entree,0.003133,0.001659,117,37230,802,482637
plat,0.004891,0.003851,183,37230,1866,482637
dessert,0.000671,0.008859,25,37230,4314,482637


In [149]:
#Création du fichier excel
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

writer = pd.ExcelWriter('results.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='results')
writer.save()